In [5]:
import pandas as pd;
import numpy as np;
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest
import scipy.stats as stats

plt.rcParams.update({'font.size': 14})
pd.set_option('display.float_format', lambda x: '%.2f' % x)
path = '../../../../playwright/results/core-web-vitals/testrun-8/'


In [6]:
df = pd.read_csv(f'{path}2024-12-15T22:05:17.080Z_results-ssrh.csv', sep=',')
df_t = pd.read_csv(f'{path}2024-12-15T23:30:49.336Z_results-ssrh-throttled.csv', sep=',')
df_ht = pd.read_csv(f'{path}2024-12-16T07:53:32.348Z_results-ssrh-throttled-heavy.csv', sep=',')

features = ['navTime', 'totalTime', 'lcp', 'fcp', 'ttfb', 'tbt', 'tti', 'longestTask', 'longTasks', 'nf:init', 'nf:config','nf:loaded']

dirty_dfs = {
    'unthrottled': df[5:],
    'throttled': df_t[5:],
    'heavy': df_ht[5:],
}

In [7]:
def detect_outliers(_df, _features, contamination=0.1):
    clf = IsolationForest(contamination=contamination, random_state=42)
    outliers = clf.fit_predict(_df[_features])
    return outliers == 1

masks = {}
dfs = {}
target_features = ['navTime', 'totalTime', 'lcp', 'fcp', 'ttfb']

for name, _df in dirty_dfs.items():
    mask = detect_outliers(_df, target_features)
    masks[name] = mask
    dfs[name] = _df[mask].copy()

In [8]:
columns = [ 'ttfb','fcp','nf:init','lcp','tti','nf:loaded','tbt','longestTask']
rows = []

for name, df in dfs.items():
    mean_row = df[columns].mean()
    percentile_row = df[columns].quantile(0.75)
    
    rows.append((f"{name} (mean)", mean_row))
    rows.append((f"{name} (75th)", percentile_row))

result_df = pd.DataFrame([row[1] for row in rows], index=[row[0] for row in rows])
result_df = result_df.mask(result_df < 0, '-')
result_df

,ttfb,fcp,nf:init,lcp,tti,nf:loaded,tbt,longestTask
unthrottled (mean),16.36,89.71,55.73,89.71,89.71,112.62,0.00,-
unthrottled (75th),17.40,91.57,58.37,91.57,91.57,115.90,0.00,-
throttled (mean),369.79,1189.97,1482.76,1189.97,3510.37,3719.97,16.09,66.09
throttled (75th),370.60,1193.90,1487.50,1193.90,3515.20,3723.40,17.00,67.00
heavy (mean),370.79,1261.85,1590.22,1261.85,3969.91,3970.23,171.63,169.08
heavy (75th),371.80,1265.30,1594.85,1265.30,3976.30,3976.80,173.00,170.00
